In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git accelerate>=0.16.0 torchvision transformers>=4.25.1 ftfy tensorboard Jinja2 datasets wandb

In [ ]:
!huggingface-cli login

In [ ]:
!wandb login


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
!git clone https://github.com/katryo/jupyter-train-controlnet-sdxl.git

In [ ]:
from google.colab import drive
import os

root_dir          = "/content"
drive_dir         = os.path.join(root_dir, "drive/MyDrive")
training_dir      = os.path.join(root_dir, "fine_tune")

def mount_drive(dir):
    output_dir      = os.path.join(training_dir, "output")

    if not os.path.exists(drive_dir):
        drive.mount(os.path.dirname(drive_dir))
    output_dir  = os.path.join(drive_dir, "kohya-trainer/output")

    return output_dir


output_dir      = mount_drive(drive_dir)

for dir in [training_dir, output_dir]:
    os.makedirs(dir, exist_ok=True)

In [ ]:
!export MODEL_DIR="stabilityai/stable-diffusion-xl-base-1.0"
!export OUTPUT_DIR="/content/fine_tune/output"

!accelerate launch train_controlnet_sdxl.py \
 --pretrained_model_name_or_path=$MODEL_DIR \
 --output_dir=$OUTPUT_DIR \
 --dataset_name=katryo/mkcface-depth-0 \
 --mixed_precision="fp16" \
 --resolution=1024 \
 --learning_rate=1e-5 \
 --max_train_steps=15000 \
 --tracker_project_name "mkcface-depth-0" \
 --validation_image "./conditioning_image_1.png" "./conditioning_image_2.png" \
 --validation_prompt "red circle with blue background" "cyan circle with brown floral background" \
 --validation_steps=100 \
 --train_batch_size=1 \
 --gradient_accumulation_steps=4 \
 --report_to="wandb" \
 --seed=42 \
 --push_to_hub